# Issues / Improvements with OL

The current benchmark is [vanilla mapbox](http://vectortiles.geo.admin.ch/gl-styles/ch.swisstopo.leichte-basiskarte.vt/v006/) viewer for vectortiles. The "model under test" is [vanilla OL](https://mf-geoadmin4.int.bgdi.ch/ol-minimalist/index.html). The style files are slightly modified to use only a single source host (`vectortiles100.geo.admin.ch`) instead of a range of five hosts (performance improvement on browsers supporting http/2).


## Performance

- [time of loading new tiles](#Timing-of-loading-new-tiles)
    
- [rendering](#Rendering)
    
- [caching](#Caching)

## Documentation

- API documentation is very good
- "higher" level doc of concepts / architecture (e.g. call tree) is missing

## Roadmap

- in [ol wiki](https://github.com/openlayers/openlayers/wiki/OpenLayers-6:-call-for-funding) it is written that mapbox-gl should be integrated as ol-layer, status?
- how should several VT sources / layers be integrated in an app
    - basemap from one source with 5 vt-layers
    - additional source with 1-X vt-layers
    - raster layer

### Timing of loading new tiles

When recording performance of a mouse scroll zoom action it becomes apparent that the timing of loading new data is only when the zoom-animation has finished. 

![vanilla ol](img/ol-late-tile-loading.png)
*vanilla ol*

Comparing this to mapbox reveals that in MB, loading newer data is fired right after the zoom event has occured. Thus, tile loading has finished once animation has finished and rendering of the new data can immediately start. In what ways can this behaviour be improved in OL?

Ideas herefore could be:
- start loading of new tiles immediately: is this feasible, what would have to be changed in the architecture to not interfere with animation
- how is it determined, when new data has to be loaded? can this made predictive (e.g. panning left immediately start loading new tiles on the right of the map)

### Rendering

#### Zoom steps

Zoom / pan animation is not done fully "vectorial" but based on rastered images that are zoomed. This results in several visual side effects
- feature size doesn't change smootly between zoom levels: features are enlarged too much by zooming the rastered image, subsequent rerendering reduces the feature size to the correct value. Vector zooming would allow for a smooth transition between feature sizes (this behaviour can be best observed at roundabouts).
- "fading in" of features when zooming is not working as intended (best observed in forest areas), MB zoom smoothly changes the forest color whereas OL zoom animation uses the "old" color and then abruptly changes to the new color.

Zooming by mouse wheel is currently too coarse (zooming step of one per wheel 'tick'), can be reduced to 0.33 (as in Mapbox) by reducing the `MAX_DELTA` constant (in `interaction/MouseWheelZoom.js:17`
```interaction/MouseWheelZoom.js:17
/**
 * Maximum mouse wheel delta.
 * @type {number}
 */
const MAX_DELTA = 0.3;
```

Furthermore, there is a difference in mouse wheel zooming and trackpad zooming. Trackpad shows a smoother zooming behaviour (more frequent re-rendering) than mouse wheel zooming. Performance recording shows that trackpad zooming consists of numerous small steps with intermediate re-rendering while mouse-wheel zooming is one long animation and subsequent re-rendering.

#### Performance

The overall rendering performance can't keep up with mapbox: what is the status of using webworkers and webgl? performance analysis shows that MB is extensively using GPU, why did OL come to the conclusion that it is not performant?

Note: the visual effect of the described behaviours can be slightly reduced by tweaking zoom steps and animation times.

### Caching

It seem that caching strategy of tiles could be improved. This is most apparent when zooming out of a region that was previously already displayed at a lower zoom level. Checking network traffic reveals that tiles are loaded again that have been previously loaded (although most of them from browser cache). Again, MB doesn't show this behaviour, when zooming in and out again at the exact same point, no tiles are reloaded.

asdfasdf
